## Day 18: Boiling Boulders

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day18/puzzle.ipynb)

### Preparation

For the puproses of visualizing it is worth modelling the cubes as a 3D matrix which elements describe which faces the corresponding cubes have open.

A face can de represented by one of six axis-aligne 3D vectors, showing the direction in which the face is located relative to the cube.

In [99]:
#load "../common/common.fsx"

type Face = Point3

let faces = 
    [
        0,1,0
        1,0,0
        0,0,1
        0,-1,0
        -1,0,0
        0,0,-1
    ] |> List.map (Face.Point3)

type Cube = { OpenFaces: Face list }

In [100]:
let private parsePoint s = 
    match s with
    | Regex "(\d+),(\d+),(\d+)" [ x; y; z ] -> Point3(int x, int y, int z)

let buildMap input = 
    let points = Pattern1.read parsePoint input
    let (maxx,maxy,maxz) = points |> Array.fold (fun (x,y,z) (Point3(px,py,pz)) -> (max x px),(max y py),(max z pz)) (0,0,0)
    let map = Array3D.create (maxx+1) (maxy+1) (maxz+1) None
    points |> Array.iter (fun (Point3(x,y,z)) -> map[x,y,z] <- Some { OpenFaces = [] })
    let isFaceEmpty p face =
        match Array3D.tryAtPoint (p+face) map with
        | None -> true
        | Some (None) -> true
        | _ -> false
    map 
    |> Array3D.mapi(fun x y z value -> 
        match value with
        | None -> None
        | Some _ -> 
            let p = Point3(x,y,z)
            Some { OpenFaces = faces |> List.filter (fun face -> isFaceEmpty p face) } 
    )


For visualization, since we know exaclty which faces are open, only these faces can be displayed, not complete cubes. Additionaly, combining all the faces in a single mesh instead of having many meshes in a single chart will likely provide the best performance

In [101]:
#r "nuget: Plotly.NET.Interactive, 3.0.2"
open Plotly.NET

Installed Packages Plotly.NET.Interactive, 3.0.2

In [102]:
type Mesh = { X: int list; Y: int list; Z: int list; i: int list; j: int list; k: int list}

let private mesh3d color (mesh : Mesh) = 
    Chart.Mesh3D(
        mesh.X, mesh.Y, mesh.Z,
        mesh.i, mesh.j, mesh.k,
        FlatShading = true,
        Color = color,
        Opacity = 0.3
    )

let private meshFromPoints (points : (Point3*Point3*Point3) list) = 
    let all = points |> Seq.collect (fun (p1,p2,p3) -> seq {p1;p2;p3}) |> Seq.distinct |> List.ofSeq
    let x = all |> List.map Point3.x
    let y = all |> List.map Point3.y
    let z = all |> List.map Point3.z
    let i = points |> List.map (fun (i,_,_) -> all |> List.findIndex ((=)i))
    let j = points |> List.map (fun (_,j,_) -> all |> List.findIndex ((=)j))
    let k = points |> List.map (fun (_,_,k) -> all |> List.findIndex ((=)k))
    { X = x; Y = y; Z = z; i = i; j = j; k = k}

let cubesToMesh color cubes = 
    let allPoints = 
        [|
            0,0,0
            0,0,1
            1,0,1
            1,0,0
            0,1,0
            0,1,1
            1,1,1
            1,1,0
        |] |> Array.map (Point3) |> Array.toList
    let ijkForFace face = 
        match face with
        | Point3(0,0,1) -> [1,2,5;2,5,6]
        | Point3(0,1,0) -> [5,4,6;4,6,7]
        | Point3(1,0,0) -> [2,3,6;3,6,7]
        | Point3(0,0,-1) -> [0,3,4;3,4,7]
        | Point3(0,-1,0) -> [1,2,0;2,0,3]
        | Point3(-1,0,0) -> [1,0,5;0,5,4]
    cubes  // indexed?
    |> Array3D.mapi (fun x y z value -> 
        match value with
        | None -> None
        | Some { OpenFaces = openFaces } -> 
            let allPoints1 = 
                allPoints 
                |> List.map (fun pp -> pp + Point3(x,y,z))
            let ijk = openFaces |> List.collect ijkForFace
            ijk 
            |> List.map (fun (i,j,k) -> allPoints1[i], allPoints1[j], allPoints1[k])
            |> Some
    ) |> Array3D.toSeq
    |> Seq.choose id
    |> Seq.collect id
    |> Seq.toList
    |> meshFromPoints
    |> mesh3d color

### Part 1

In [103]:
#!value --name sampleRaw
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5

In [104]:
#!share sampleRaw --from value

let sampleMap = sampleRaw |> buildMap

sampleMap |> cubesToMesh (Color.fromKeyword(ColorKeyword.Grey))

<!-- Plotly chart will be drawn inside this DIV -->

Counting up all the sides that aren't connected to another cube is trivial since we already have this information in the map.

In [105]:
let countFaces map =
    map
    |> Array3D.toSeq
    |> Seq.choose id
    |> Seq.map (fun cube -> cube.OpenFaces |> List.length)
    |> Seq.sum

sampleMap |> countFaces

64

For the actual input:

In [106]:
#!value --name actualRaw --from-file ./data_actual.txt

In [107]:
#!time 
#!share actualRaw --from value 
let actualMap = actualRaw |> buildMap
actualMap |> countFaces

4244

Wall time: 17.9674ms

In [108]:
actualMap |> cubesToMesh (Color.fromKeyword(ColorKeyword.Grey))

<!-- Plotly chart will be drawn inside this DIV -->

### Part 2

For part 2, cubes with faces that are exterior can be traversed with the BF-search from cube (0,0,0). Not only the cubes, but also the faces shared by them and the previous (`None`) cubes.

The algorithm will be able to walk a little outside the 3D array.

To calculate face vectors when reaching a cube, we can simply substract two cubes' coordinates.


In [109]:
#load "../common/bfs.fsx"

open Bfs
open Bfs.Custom

type private State = { Coordinates: Point3; Cube: Cube option; OriginalMap: (Cube option)[,,] }

let private target : Target<State> = 
    fun state -> false

let private adj : Adjacency<State> = 
    fun state ->
        match state.Cube with
        | None ->
            let adjCubes = faces |> List.map ((+)state.Coordinates)
            adjCubes 
            |> List.collect (fun (Point3(i, j, k) as a) -> 
                if (i < -1 || j < -1 || k < -1
                    || i > Array3D.length1 state.OriginalMap
                    || j > Array3D.length2 state.OriginalMap
                    || k > Array3D.length3 state.OriginalMap)
                then []
                else 
                    match state.OriginalMap |> Array3D.tryAtPoint a with
                    | None | Some(None) -> [ { state with Coordinates = a; Cube = None } ]
                    | Some (Some _) -> 
                        [ { state with Coordinates = a; Cube = Some ({ OpenFaces = [state.Coordinates-a] }) } ]
            )
        | Some _ -> []


let private settings = { VisitedKey = fun state -> state.Coordinates, state.Cube }

let exterior map =
    let (NotFound(paths)) = findPath settings { Adjacency = adj } {Coordinates = Point3(0,0,0); Cube = None; OriginalMap = map} target
    let newMap = Array3D.create (Array3D.length1 map) (Array3D.length2 map) (Array3D.length3 map) None
    paths |> List.map (List.head) 
    |> List.choose (fun s -> match s.Cube with | Some cube -> Some (s.Coordinates,cube) | None -> None)
    |> Seq.group (fun (p,cube) -> p) (fun values -> 
        let faces = values |> Seq.collect(fun (p,cube) -> cube.OpenFaces) |> List.ofSeq
        { OpenFaces = faces }
    )
    |> Seq.iter (fun (Point3(i,j,k),cube) -> newMap[i,j,k] <- Some cube)
    newMap

In [110]:
#!time

exterior sampleMap
|> countFaces

58

Wall time: 17.3859ms

For the actual input:

In [111]:
#!time

exterior actualMap
|> countFaces

2460

Wall time: 47.2657ms

In [112]:
let substract (cubes2 : (Cube option) [,,]) (cubes1 : (Cube option) [,,]) = 
    cubes2
    |> Array3D.mapi (fun i j k cube2 -> 
        let cube1 = cubes1[i,j,k]
        match cube2, cube1 with
        | Some { OpenFaces = openFaces2 }, Some { OpenFaces = openFaces1 } ->
            let newFaces = Seq.except openFaces1 openFaces2 |> List.ofSeq
            Some { OpenFaces = newFaces }
        | Some { OpenFaces = openFaces2 }, None -> 
            Some { OpenFaces = openFaces2 }
        | None, _ -> None
    )

In [113]:
[
    substract sampleMap (exterior sampleMap) |> cubesToMesh (Color.fromKeyword(ColorKeyword.Blue))
    exterior sampleMap |> cubesToMesh (Color.fromKeyword(ColorKeyword.Grey))
] |> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

In [114]:
[
    substract actualMap (exterior actualMap) |> cubesToMesh (Color.fromKeyword(ColorKeyword.Blue))
    exterior actualMap |> cubesToMesh (Color.fromKeyword(ColorKeyword.Grey))
] |> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->